In [1]:
from pennylane import numpy as np
import pennylane as qml
from kernels import *
from tqdm.notebook import tqdm

In [2]:
def make_gradient_kernel(kernel):
    def f(params):
        return kernel(params[0], params[1])
    
    def grad_f(params):
        params = np.array(params, requires_grad=True)
        return qml.jacobian(f)(params)
    
    return grad_f

def variance(f, params, N):
    return np.var([f(params) for _ in range(N)])

def variance_over_range(f, N, extent=2 * np.pi):    
    values = tqdm([(x, y) for x in np.linspace(0, extent, N) for y in  np.linspace(0, extent, N)])
    
    res = np.array(list(ap(lambda p: variance(f, p, 10), values)))
    return np.var(res)

In [3]:
trial_kernels = {
    'XYZ': K_layered,
    'L-prod': K_L_prod,
    'L-tower': K_L_tower,
    'L-cheb': K_L_cheb,
}

for (label, kernel) in trial_kernels.items():
    k = kernel(8, n_layers=2, qml_backend="qiskit.aer", diff_method="parameter-shift")
    f = make_gradient_kernel(k)
    print(f"Variance of {label} kernel: {variance_over_range(f, 10)}")

  0%|          | 0/100 [00:00<?, ?it/s]

NameError: name 'ap' is not defined